In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

import numpy as np
from numpy import random
from time import sleep

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.nytimes.com/games/wordle/index.html')

actions = ActionChains(driver)
actions.click()
actions.perform()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Benjamin\.wdm\drivers\chromedriver\win32\100.0.4896.60]
C:\Users\Benjamin\AppData\Local\Temp\ipykernel_14068\4029482767.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
wordleGuesses = np.loadtxt('wordle-guesses.txt', dtype = str)

def make_random_guess():
    guessIdx = np.random.randint(low = 0, high = len(wordleGuesses))
    guess = wordleGuesses[guessIdx]
    actions.send_keys(guess)
    actions.send_keys(Keys.RETURN)
    actions.perform()

def evaluate_random_guess(idx):
    """
    
    
    """

    game_app = driver.find_element(By.TAG_NAME , 'game-app')
    game_rows = driver.execute_script("return arguments[0].shadowRoot.getElementById('board')", game_app).find_elements(By.TAG_NAME, 'game-row')
    letters = driver.execute_script('return arguments[0].shadowRoot', game_rows[idx]).find_elements(By.CSS_SELECTOR , 'game-tile')

    # Quantize evaluation 
    eval_to_int = {
        'correct' : 2,
        'present' : 1,
        'absent'  : 0
    }

    # Evaluate guess
    correctness = 0
    for letter in letters:
        correctness += eval_to_int[letter.get_attribute('evaluation')]
    correctness /= 10
    print('Correctness: {:.2f}'.format(correctness))

In [4]:
state = np.loadtxt('wordle-guesses.txt', dtype = str)

In [24]:
# Interpret gameboard
game_app = driver.find_element(By.TAG_NAME , 'game-app')
game_rows = driver.execute_script("return arguments[0].shadowRoot.getElementById('board')", game_app).find_elements(By.TAG_NAME, 'game-row')
game_tiles = driver.execute_script('return arguments[0].shadowRoot', game_rows[1]).find_elements(By.CSS_SELECTOR , 'game-tile')

In [49]:
correct = []
present = []
absent = []
# Parse pattern
for i, tile in enumerate(game_tiles):
  letter = tile.get_attribute('letter')
  eval = tile.get_attribute('evaluation')
  # Letter is present and at exact position in answer
  if eval == 'correct':
    # Add words to new state with letter at POSITION `i` in word
    correct.append([word for word in state if word[i] == letter])
  # Letter is present in answer
  elif eval == 'present':
    # Add words to new state WITH LETTER in word
    present.append([word for word in state if letter in word])
  # Letter is not present in answer
  else:
    print(letter)
    # Add words to new state WITHOUT LETTER in word
    absent.append([word for word in state if letter not in word ])
# New worde state is INTERSECTION of three states; (correct x present x absent)
# new_state = np.array(list(set.intersection(*map(set, [correct, present, absent]))))

n
k


In [53]:
# has 't' at position 0
correct2 = list(set.intersection(*map(set, correct)))
# has both 'a' and 's'
present2 = list(set.intersection(*map(set, present)))
# does not have either 'n' or 'k'
absent2 = list(set.intersection(*map(set, absent)))
# has 't' at position 0 and contains both 'a' and 's' and does not contain 'n' or 'k'
intersection = list(set.intersection(*map(set, [correct2, present2, absent2])))

In [77]:
# CORRECT WORD: 'TRAIN'
# ---------------------
# Correct
#   -> ('t' @ idx 0)
#   -> ('r' @ idx 1)
# Must satsify t @ idx 0 AND r @ idx 1
s = [['train', 'tank', 'tent'], ['train', 'brain', 'pray']] 
# Present
#   -> ('a')
#   -> ('i')
# Must satsify 'a' in word AND 'i' in word
t = [['train', 'apple', 'air'], ['brain', 'train', 'item']]
# Not in
u = []
sets = [s, t, u]
for i in np.arange(3):
    subset = sets[i]
    if subset:
        sets[i] = list(set.intersection(*map(set, subset)))
s, t, u = tuple(sets)

sets = [subset for subset in [s, t, u] if subset]
list(set.intersection(*map(set, sets)))

['train']